# 6.21.22 Data Analysis

Today we will extract ATP gradients from an aster dataset. Executing the next few cells will import all the relevant data.

## Importing Pkg

In [1]:
# Packages to locally download data into the notebook:
import requests
import zipfile
import glob

#Numpy
import numpy as np

#For image plotting
import skimage.io

#Matplotlib plotting packages
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

#For identifying aster center
from skimage.filters import threshold_otsu, gaussian
from skimage.measure import regionprops
import cv2

#for image registration
from skimage.registration import phase_cross_correlation
import os

# For loading bars
from tqdm.notebook import tqdm as tqdm

## Importing plotting

In [3]:
import plotting
import celluloid as cell

In [4]:
import matplotlib.animation as animation

## Initializing Images

In [5]:
def file_to_image(files):
    """
    Takes in a file list and converts to images.
    
    Parameters:
    files: list of file names that you want to read into images
    
    Returns:
    An array of np.int16 images
    """
    im_list = list()
    for file in files:
        im = skimage.io.imread(file)
        im_list.append(im.astype(np.int16))

    return np.array(im_list)

### Specifying files

In [6]:
#Choose the experiment file name(located in folder 'Asters'):
file_path = '../Users/cathji/Dropbox (MIT)/wave 2022/aster_data'

# #Choose the position(s)/aster(s) of interest:
pos_list = ['Pos0']

#Choose the channels of interest (this is almost always going to be Cherry and iATP)
channel_list = ['Bright', 'CFP', 'YFP','Cherry']

#included frames
included_frames = '*2022*.tif'

### Creating image arrays

In [7]:
# pathname: /Users/cathji/Dropbox (MIT)/wave 2022/aster_data/2022-06-20_Ncd_ATeam_1_MMStack_Pos0.ome.tif

# Import images
for pos in pos_list:
    for channel in channel_list:

        #creates an empty list with the naming convention
        exec('im_files' + channel + pos + '= []')
        im_files = eval('im_files' + channel + pos)
        
        #appends all the file names
        im_files.append(
            np.sort(
                glob.glob(file_path + '/' + pos + '/' + channel + '/' +
                          included_frames)))

        #Reads the files as images
        exec('im_array' + channel + pos + ' = file_to_image(im_files[0])')
        print('im_array' + channel + pos)

im_arrayBrightPos0
im_arrayCFPPos0
im_arrayYFPPos0
im_arrayCherryPos0


### Reading bg images

In [8]:
# Camera dark images - for background subtraction

im_MT_dark = skimage.io.imread(
    '/Users/cathji/Dropbox (MIT)/wave 2022/aster_data/Dark/C1-2022-06-13_Ncd_ATeam_Victor+cathy_fullframe_2_MMStack_Pos1.ome.tif'
).astype(np.int16)

im_YFP_dark = skimage.io.imread(
    '/Users/cathji/Dropbox (MIT)/wave 2022/aster_data/Dark/C2-2022-06-13_Ncd_ATeam_Victor+cathy_fullframe_2_MMStack_Pos1.ome.tif'
).astype(np.int16)

im_CFP_dark = skimage.io.imread(
    '/Users/cathji/Dropbox (MIT)/wave 2022/aster_data/Dark/C3-2022-06-13_Ncd_ATeam_Victor+cathy_fullframe_2_MMStack_Pos1.ome.tif'
).astype(np.int16)

im_Cherry_dark = skimage.io.imread(
    '/Users/cathji/Dropbox (MIT)/wave 2022/aster_data/Dark/C4-2022-06-13_Ncd_ATeam_Victor+cathy_fullframe_2_MMStack_Pos1.ome.tif'
).astype(np.int16)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/cathji/Dropbox (MIT)/wave 2022/aster_data/Dark/C1-2022-06-13_Ncd_ATeam_Victor+cathy_fullframe_2_MMStack_Pos1.ome.tif'

### Param calibration

In [ ]:
#Length Calibration (microns to pixels)
um_per_pixel = 0.59

#Image dims
w_px, h_px = 1920, 1200  #pixels
w_um, h_um = w_px * um_per_pixel, h_px * um_per_pixel  # microns

#min area thresh (IN PIXELS)
area_thresh = 5000

#Fontsize
f_small, f_big = 13, 15

#timestamps (IN FRAMES)
total_frames = 200
T_min = 20
T_span = total_frames - T_min

#??
winsize = 100
thickness = 5
scale = 5

In [ ]:
frame = 3
im_MT = im_arrayBrightPos0[frame]
im_CFP = im_arrayCFPPos0[frame]
im_YFP = im_arrayYFPPos0[frame]

#define colorbar max and min
colorbar_max = np.percentile(im_YFP, 100)
colorbar_min = np.percentile(im_CFP, 0)

#define plot
fig, ax = plt.subplots(2, 3, figsize=(16, 8))
cmap = cm.get_cmap('viridis')
normalizer = Normalize(colorbar_min, colorbar_max)
cb_setting = cm.ScalarMappable(norm=normalizer)

#define each subplot
ax[0, 0].imshow(im_MT)
ax[0, 0].set_title(r'MT (all units in pixels)', fontsize=f_big)
ax[0, 0].set_ylabel(r'Test Image', fontsize=f_small)

ax[0, 1].imshow(im_YFP)
ax[0, 1].set_title(r'YFP (all units in pixels)', fontsize=f_big)

ax[0, 2].imshow(im_CFP)
ax[0, 2].set_title(r'CFP (all units in pixels)', fontsize=f_big)

#subplots: setting bg plots
ax[1, 0].imshow(im_MT)
ax[1, 0].set_ylabel(r'Background Image', fontsize=f_small)

ax[1, 1].imshow(im_YFP)

ax[1, 2].imshow(im_CFP)

fig.colorbar(cb_setting, ax=ax.ravel().tolist())

### Subtracting bg image

In [ ]:
# subtracting bg image

im_arrayBrightPos0_bs = im_arrayBrightPos0 - im_MT_dark
im_arrayYFPPos0_bs = im_arrayYFPPos0 - im_YFP_dark
im_arrayCFPPos0_bs = im_arrayCFPPos0 - im_CFP_dark

In [ ]:
# test

plt.imshow(im_arrayBrightPos0_bs[30])
plt.colorbar()

### Crop

In [ ]:
#set crop pixel values
tc_ymin, tc_ymax, tc_xmin, tc_xmax = 475, 655, 870, 1040 #for 'tight crop'
#define crop window
tight_crop = np.s_[tc_ymin:tc_ymax, tc_xmin:tc_xmax]

#set crop pixel values
wc_ymin, wc_ymax, wc_xmin, wc_xmax = 515, 615, 900, 1010  #for 'tight crop'
#define crop window
wide_crop = np.s_[wc_ymin:wc_ymax, wc_xmin:wc_xmax]

#show the crop
plt.imshow(im_arrayBrightPos0_bs[30][tight_crop])
plt.colorbar()

Find the center for each image

### Center

In [ ]:
c_ymin, c_ymax, c_xmin, c_xmax = 100, 900, 700, 1500
# define crop window
crop = np.s_[c_ymin:c_ymax, c_xmin:c_xmax]

In [ ]:
#init empty lists for center vals, radius, frame #
centers_tc = []
radii = []
frame_num = []
contours_plot = []

#for loop thru MT images
for i, im in enumerate(im_arrayBrightPos0_bs):
    #crop image for center identification
    image = im

    #perform otsu threshholding
    thresh = threshold_otsu(image)

    #create binary image
    binary = (image > thresh).astype(np.uint8)

    #create a contour
    contours, hierarchy = cv2.findContours(binary, 1, 2)

    #iterate through contours (for most connected region)
    max_area = 0
    max_index = None

    for j, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area > max_area and 100000 >= max_area:
            max_area = area
            max_index = j

    if 100000 >= max_area >= area_thresh:
        cnt = contours[max_index]
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        centers_tc.append([x, y])
        radii.append(radius)
        frame_num.append(i)
        contours_plot.append(cnt)
    else:
        print("**CENTER NOT FOUND**")
        cnt = contours[max_index]
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        centers_tc.append([0, 0])
        radii.append(radius)
        frame_num.append(i)
        contours_plot.append(cnt)
        
centers_tc = np.array(centers_tc)
radii = np.array(radii)
frame_num = np.array(frame_num)

### Center (w/ Gaussian blur)

In [ ]:
c_ymin, c_ymax, c_xmin, c_xmax = 100, 900, 700, 1500

In [ ]:
im = im_arrayBrightPos0_bs[30]

dst = gaussian(im[crop], sigma=50)
thresh = np.percentile(dst, 99)
binary = (dst>thresh).astype(np.uint8)

#create a contour
contours, hierarchy = cv2.findContours(binary, 1, 2)

In [ ]:
cnt = np.array(contours)
cnt = np.squeeze(cnt)

In [ ]:
cnt[:,0] = cnt[:,0] + c_xmin
cnt[:,0]

In [ ]:
#init empty lists for center vals, radius, frame #
centers_tc = []
radii = []
frame_num = []
contours_plot = []

#for loop thru MT images
for i, im in tqdm(enumerate(im_arrayBrightPos0_bs)):
    #crop image for center identification

    #gaussian blur
    dst = gaussian(im[crop], sigma=50)
    thresh = np.percentile(dst, 99)
    binary = (dst>thresh).astype(np.uint8)
    
    #create a contour
    contours, hierarchy = cv2.findContours(binary, 1, 2)

    #iterate through contours (for most connected region)
    max_area = 0
    max_index = None

    for j, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_index = j

    if max_area >= area_thresh:
        cnt = contours[max_index]
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        centers_tc.append([x + c_xmin, y + c_ymin])
        radii.append(radius)
        frame_num.append(i)
        
        cnt = np.array(cnt)
        cnt = np.squeeze(cnt)
        cnt[:,0] += c_xmin
        cnt[:,1] += c_ymin
        contours_plot.append(cnt)
        
    else:
        print("**CENTER NOT FOUND**")
        cnt = contours[max_index]
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        centers_tc.append([0, 0])
        radii.append(radius)
        frame_num.append(i)
        
        cnt = np.array(cnt)
        cnt = np.squeeze(cnt)
        cnt[:,0] += c_xmin
        cnt[:,1] += c_ymin
        contours_plot.append(cnt)
        
centers_tc = np.array(centers_tc)
radii = np.array(radii)
frame_num = np.array(frame_num)

Find what the center vcalues are for full sized image

In [ ]:
centers = []
for center in centers_tc:
    center_x = int(center[0])
    center_y = int(center[1])
    centers.append([center_x, center_y])
centers = np.array(centers)

In [ ]:
#TEST: plotting all centers

plt.figure()

for i in range(np.shape(centers)[0]):
    plt.scatter(centers[i, 0], centers[i, 1], color='r')

plt.xlim(800, 1200)
plt.ylim(400, 600)

## Making video

In [ ]:
#TEST: checking ALL FRAMES centers (only run once please)
ims = []
cmap = cm.get_cmap('viridis')
normalizer = Normalize(colorbar_min, colorbar_max)
cb_setting = cm.ScalarMappable(norm=normalizer)
fig, ax = plt.subplots()

ax.set_xlim([200,1700])
ax.set_ylim([0,1200])
ax.set_title("Detecting Border")

camera = cell.Camera(fig)

for i in tqdm(range(total_frames)):
    im_temp = im_arrayBrightPos0_bs[i]
    im = ax.imshow(im_temp,
          vmin=np.percentile(im_temp, 0),
          vmax=np.percentile(im_temp, 100),
          animated=True,
          zorder = 0)
    
    ax.scatter(contours_plot[i][:, 0],
               contours_plot[i][:, 1],
               color='r',
               s=0.2)
    ax.scatter(centers_tc[i, 0], centers_tc[i, 1], color='black', s = 0.2)

    circle1 = plt.Circle((centers[i, 0], centers[i, 1]),
                         radii[i],
                         fill=False,
                         color='w')
    ax.add_patch(circle1)
    camera.snap()
    
animation = camera.animate()
tqdm(animation.save("movie.mp4"))

## Register

Register images

In [ ]:
#Empty list to record how much CFP (probe) channel is shifted w.r.t. YFP (ATP is bound) channel
shifts = []

for i in range(total_frames):
    shift, error, _ = phase_cross_correlation(
        im_arrayYFPPos0_bs[i][tight_crop], im_arrayCFPPos0_bs[i][tight_crop])
    shifts.append(shift)

shifts = np.array(shifts)

## Analyzing Images

In [ ]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
        
    mask = dist_from_center <= radius
    return mask

In [ ]:
#cropped aster images
im_arrayYFPPos0_plot = []
im_arrayCFPPos0_plot = []
im_arrayBrightPos0_plot = []
ratios_plot = []

#Traces for asters
YFP_trace_pts = []
CFP_trace_pts = []
MT_trace_pts = []
ratios_trace_pts = []

for j in range(T_span):

    i = j + T_min
    #Set the window parameters
    i_min = int(centers[i][0] - winsize)
    i_max = int(centers[i][0] + winsize)
    j_min = int(centers[i][1] - winsize)
    j_max = int(centers[i][1] + winsize)

    #demi-variable for shift
    shift_j = int(shifts[i][0])
    shift_i = int(shifts[i][1])

    #set YFP and CFP windows
    winYFP = np.s_[j_min:j_max, i_min:i_max]
    winCFP = np.s_[(j_min - shift_j):(j_max - shift_j),
                   (i_min - shift_i):(i_max - shift_i)]

    #Append the plotting images
    [h, w] = np.shape(im_arrayYFPPos0_bs[i])
    mask = 1 * create_circular_mask(h,w,centers[i],thickness) 
    
    im_arrayYFPPos0_plot.append(im_arrayYFPPos0_bs[i][winYFP])
    im_arrayCFPPos0_plot.append(im_arrayCFPPos0_bs[i][winCFP])
    im_arrayBrightPos0_plot.append(im_arrayBrightPos0_bs[i][winCFP])
    #for now MT same as CFP shifts, running the shift cell seems closest to CFP

    ratio = im_arrayYFPPos0_bs[i][winYFP] / im_arrayCFPPos0_bs[i][winCFP]
    ratios_plot.append(ratio)

    #Define a narrow srip to take trace values
    YFP_trace = im_arrayYFPPos0_plot[j][(winsize - thickness):(winsize +
                                                               thickness), :]
    CFP_trace = im_arrayCFPPos0_plot[j][(winsize - thickness):(winsize +
                                                               thickness), :]
    MT_trace = im_arrayBrightPos0_plot[j][(winsize - thickness):(winsize +
                                                                 thickness), :]
    ratios_trace = ratios_plot[j][(winsize - thickness):(winsize +
                                                         thickness), :]

    #average the strips
    YFP_trace_pts.append(np.average(YFP_trace, axis=0))
    CFP_trace_pts.append(np.average(CFP_trace, axis=0))
    MT_trace_pts.append(np.average(MT_trace, axis=0))
    ratios_trace_pts.append(np.average(ratios_trace, axis=0))

im_arrayYFPPos0_plot = np.array(im_arrayYFPPos0_plot)
im_arrayCFPPos0_plot = np.array(im_arrayCFPPos0_plot)
im_arrayBrightPos0_plot = np.array(im_arrayBrightPos0_plot)
ratios_plot = np.array(ratios_plot)

YFP_trace_pts = np.array(YFP_trace_pts)
CFP_trace_pts = np.array(CFP_trace_pts)
MT_trace_pts = np.array(MT_trace_pts)
ratios_trace_pts = np.array(ratios_trace_pts)

pos_xaxis = np.linspace(-winsize, winsize, 2 * winsize)
actual_frame = np.arange(T_min * scale, (T_span + T_min) * scale, scale) #not sure about last param

### Plotting analysis

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4.3))
plt.gca().set_prop_cycle(
    plt.cycler(
        'color',
        plt.cm.gist_earth(np.linspace(0.1, 0.80, len(ratios_trace_pts)))))

for i, trace in enumerate(ratios_trace_pts):
    plt.plot(pos_xaxis, trace, label=str(actual_frame[i] * 20))
#     cmap = cm.get_cmap('')
#     normalizer = Normalize(colorbar_min, colorbar_max)
#     cb_setting = cm.ScalarMappable(norm=normalizer)
#     plt.colorbar()
    plt.legend(bbox_to_anchor=(1, 1), loc="upper left", title='Time (s)')

plt.xlabel('Distance from aster center ($\mu m$)')
plt.ylabel('ATP Concentration (A.U.)')
ax.set_facecolor('#E5E0D5')

In [ ]:


#TEST: checking ALL FRAMES centers (only run once please)
ims = []
cmap = cm.get_cmap('viridis')
normalizer = Normalize(colorbar_min, colorbar_max)
cb_setting = cm.ScalarMappable(norm=normalizer)
fig, ax = plt.subplots()

ax.set_xlim([200,1700])
ax.set_ylim([0,1200])
ax.set_title("Detecting Border")

camera = cell.Camera(fig)

for i in range(total_frames):
    im_temp = im_arrayBrightPos0_bs[i]
    im = ax.imshow(im_temp,
          vmin=np.percentile(im_temp, 0),
          vmax=np.percentile(im_temp, 100),
          animated=True,
          zorder = 0)
    
    ax.scatter(contours_plot[i][:, 0, 0],
               contours_plot[i][:, 0, 1],
               color='r',
               s=0.3)
    ax.scatter(centers_tc[i, 0], centers_tc[i, 1], color='black')

    circle1 = plt.Circle((centers[i, 0], centers[i, 1]),
                         radii[i],
                         fill=False,
                         color='w')
    ax.add_patch(circle1)
    camera.snap()
    
animation = camera.animate()
animation.save("movie.mp4")


In [ ]:
from PIL import Image, ImageDraw
import io

In [ ]:
images = []
   
for i in range(0, 50000, 1000):
    fig = plt.figure()
    plt.imshow(ths_big[:,:,i], vmin = 0, vmax = 1, cmap = 'Greys_r' ,alpha=1)
    plt.imshow(zs_big[:,:,i], vmin = 0, vmax = 1, cmap = 'Blues', alpha=0.5)
    plt.imshow(exclusion, vmin = 0, vmax = 1, cmap = 'Greys')
   
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    im = Image.open(img_buf)
    images.append(im)
    plt.close(fig)
    
images[0].save('LarsenC.gif', save_all=True, append_images=images[1:], optimize=False, duration=140, loop=0)